In [1]:
import os, sys, email
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
### 自然言語処理
import stanza
#stanza.download('en')
nlp = stanza.Pipeline('en')

2020-10-05 16:01:05 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-10-05 16:01:05 INFO: Use device: cpu
2020-10-05 16:01:05 INFO: Loading: tokenize
2020-10-05 16:01:05 INFO: Loading: pos
2020-10-05 16:01:07 INFO: Loading: lemma
2020-10-05 16:01:07 INFO: Loading: depparse
2020-10-05 16:01:09 INFO: Loading: sentiment
2020-10-05 16:01:10 INFO: Loading: ner
2020-10-05 16:01:11 INFO: Done loading processors!


# メールファイルからmail_dfを作成する

In [149]:
from loadFile import getFileList

# ディレクトリ 内のメールファイルを読み込む
directory_path = "/Users/taroaso/myprojects/OpenIE/trec/2005/each_dataset/3"
file_list = getFileList(directory_path)
file_list.sort()

In [150]:
### mail_df 

mail_cols = ['docno','received','isoreceived','sent','isosent','name','email','subject','id','charset','inreplyto','expires','to','cc','body']
mail_df = pd.DataFrame(index=[], columns=mail_cols)

for file in file_list:
    with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        mail = f.readlines()
    
        record={}
        body = []
        for row in mail:
            if row.startswith('docno='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['docno'] = match.group().strip('"')
            elif row.startswith('received='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['received'] = match.group().strip('"')
            elif row.startswith('isoreceived='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['isoreceived'] = match.group().strip('"')
            elif row.startswith('sent='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['sent'] = match.group().strip('"')
            elif row.startswith('isosent='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['isosent'] = match.group().strip('"')
            elif row.startswith('name='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['name'] = match.group().strip('"')
            elif row.startswith('email='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['email'] = match.group().strip('"')
            elif row.startswith('subject='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['subject'] = match.group().strip('"')
            elif row.startswith('id='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['id'] = match.group().strip('"')
            elif row.startswith('charset='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['charset'] = match.group().strip('"')
            elif row.startswith('inreplyto='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['inreplyto'] = match.group().strip('"')
            elif row.startswith('expires='):
                match = re.search(r'".+"',row)
                if match != None:
                    record['expires'] = match.group().strip('"')
            elif row.startswith('To:'):
                match = row[3:-1]
                record.setdefault('to',match)
            elif row.startswith('Cc:'):
                match = row[3:-1]
                record.setdefault('cc',match)
            else:
                body.append(row)
        record['body'] = '\n'.join(body)
    
    mail_df = mail_df.append(record, ignore_index=True)

# RDBにmailテーブルを作る

In [186]:
# RDBにmail_dfのテーブルを作成する
from db import connect
engine = connect()
mail_df.to_sql(name='mail_3',con=engine,if_exists='replace',index=None)

# 以下の2つに関する処理
  1. メンションとEntityを対応させる辞書をつくる（entity_dict）
  2. Entityテーブルのタプルとなるentity_rowsを作る（属性：message_id, entity）

In [181]:
# mail_dfのヘッダー（from, date）から，メールアドレス，送信者，送受信日を取得する

entity_rows = [] #Entityテーブルの行を格納するリスト
entity_dict = {} 
for values in mail_df.values:
    message_id = values[0]
    date = values[3]
    sender = values[5] # 送信者
    address = values[6] # 送信者のメールアドレス
    date = re.search(r'(\d{1,2} \w{3} \d{2,4})', date) # 送信日
    date = date.group()
    # 辞書に登録する
    entity_dict.setdefault(address,{address:"MAIL"})
    entity_dict.setdefault(sender,{sender:"PERSON"})
    entity_dict.setdefault(date,{date:"DATE"})
    # 行に追加する
    entity_rows.append((message_id, address))
    entity_rows.append((message_id, sender))
    entity_rows.append((message_id, date))

In [186]:
# Bodyに含まれる文をEntity linkerにかけてEntityを取得する
# entityLinkingFromBody.ipynb

# mail_dfのsubjectをEntity linkerにかけてEntityを取得する．
total_start = time.time()
count = 0

for values in mail_df.values:
    subject = values[6]
    subject = re.sub(r'(\[.+\] )','',subject)
    subject = re.sub(r'(\n\t)',' ',subject)
    subject = re.sub(r'\n{1,}',' ',subject)
    subject = re.sub(r'\t{1,}',' ',subject)
    
    part_start = time.time()

    json_res = tagme(subject)
    linked_entities = [annotation for annotation in json_res['annotations'] if annotation['rho'] > 0.3]
    if linked_entities != []:
        for le in linked_entities:
            spot = le['spot']
            entity_dict.setdefault(spot,{le['title']:le['id']}) # 辞書に登録する
            entity_rows.append((values[1],le['title'])) # 行に追加する

    count = count + 1
    print("Processing:{}%".format((count/len(bodies)) * 100))

total_time = time.time() - total_start
print("Total time:{} minutes".format(total_time/60))

import pickle

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_dict.pickle', mode='wb') as f:
    pickle.dump(entity_dict,f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_rows.pickle', mode='wb') as f:
    pickle.dump(entity_rows,f)

# 新規Entity候補の抽出
  * NERによって，Named Entityを抽出し，entity_dictに登録がなければ，新規Entity候補として，別の辞書に登録する

In [184]:
# 新規エンティティ候補を登録する辞書をentity_candidate_dictとし，行をentity_candidate_rowsとする
entity_candidate_dict = {}
entity_candidate_rows = []
for mail in mail_df.values:
    body = mail[-1]
    #body = re.sub(r'\n{2,}','\n',mail['body'])
    #body = re.sub(r'\n{1}',' ',body)
    #body = re.sub(r'( >){1,}','',body)
    #body = re.sub(r'\*{1,}','',body)
    #body = re.sub(r'(On)','. On',body)
    doc = nlp(body)
    for ent in doc.ents:
        if ent.label_ not in ('DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'CARDINAL', 'ORDINAL'):
            ent = ent.to_dict()
            if ent["text"] in list(entity_dict.keys()):
                pass
            else:
                entity_candidate_dict[ent["text"]] = ent["type"]
                entity_candidate_rows.append((mail['message_id'],ent["text"]))
print('Entity_dictに登録のない新規エンティティ候補数:{}'.format(len(entity_candidate_dict)))

Entity_dictに登録のない新規エンティティ候補数:212


## 新規エンティティ候補（メンション）をentity_dictに登録されているメンションと名寄せする（類似のメンションを検出する）
* 名寄せできたメンションとエンティティの辞書をつくる（add_entity_dict）

In [187]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [189]:
# 名寄せ
add_entity_dict = {}

for candidate in entity_candidate_dict.keys():
    threshold = 0.5
    candidate_tokens = candidate.split()
    filtered_candidate_tokens = [token for token in candidate_tokens if token not in stop_words]
    filtered_candidate_tokens = set(list(map(str.lower, filtered_candidate_tokens)))
    for spot in entity_dict.keys():
        # jaccard distance
        spot_tokens= spot.split()
        filtered_spot_tokens = [token for token in spot_tokens if token not in stop_words]
        filtered_spot_tokens = set(list(map(str.lower,filtered_spot_tokens)))
        jd = jaccard_distance(filtered_candidate_tokens, filtered_spot_tokens)
        # edit distance
        filtered_spot = ' '.join(filtered_spot_tokens)
        filtered_candidate = ' '.join(filtered_candidate_tokens)
        ed = edit_distance(filtered_spot, filtered_candidate)/max(len(filtered_spot),len(filtered_candidate))
        if min(jd,ed) < threshold:
            threshold = min(jd,ed)
            add_entity_dict[candidate] = entity_dict[spot]
print('entity_dictと名寄せできたメンション数:{}'.format(len(add_entity_dict)))

entity_dictと名寄せできたメンション数:43


## 新規エンティティ候補内で名寄せを行い，その中の1つにマッピングする
* マッピングした辞書をmapped_ne_dictとする

In [190]:
entity_candidate_list = list(entity_candidate_dict.keys() - add_entity_dict.keys())
entity_candidate_list_lower = list(map(str.lower, entity_candidate_list))
similar_dict = {}
for i, ne1 in enumerate(entity_candidate_list_lower):
    ne1_tokens = ne1.split()
    ne1_tokens = [token for token in ne1_tokens if token not in stop_words]
    ne1_strings = ' '.join(ne1_tokens)
    similar_ne = [] # 類似するentity candidateをまとめる
    for j, ne2 in enumerate(entity_candidate_list_lower):
        # jaccard distance
        ne2_tokens = ne2.split()
        ne2_tokens = [token for token in ne2_tokens if token not in stop_words]
        jd = jaccard_distance(set(ne1_tokens), set(ne2_tokens))
        # edit distance
        ne2_strings = ' '.join(ne2_tokens)
        ed = edit_distance(ne1_strings, ne2_strings)/max(len(ne1_strings),len(ne2_strings))
        if min(jd,ed) < 0.4:
            similar_ne.append(entity_candidate_list[j])
    similar_dict[entity_candidate_list[i]] = similar_ne

mapped_ne_dict = {}
for key, value_list in similar_dict.items():
    length_list = list(map(lambda x:len(x), value_list))
    idx = length_list.index(min(length_list))
    mapped_ne_dict[key] = {value_list[idx]:entity_candidate_dict[value_list[idx]]}

## 3つの辞書（entity_dict, add_entity_dict, mapped_ne_dict）を統合する

In [191]:
integrated_entity_dict = dict(**entity_dict,**add_entity_dict,**mapped_ne_dict)

In [259]:
with open("wiki-research-l/output/integrated_entity_dict.pkl","wb") as f:
    pickle.dump(integrated_entity_dict, f)

In [118]:
import pickle
with open("wiki-research-l/output/integrated_entity_dict.pkl","rb") as f:
    integrated_entity_dict = pickle.load(f)

## entity_candidate_rowsの新規エンティティ候補（メンション）を辞書と照合し，対応するエンティティと置き換え，entity_rowsに追加する

In [244]:
for row in entity_candidate_rows:
    mention = row[1]
    entity = list(integrated_entity_dict[mention].keys())[0]
    row = (row[0], entity)
    entity_rows.append(row)

# RDBにEntityテーブルを作る

In [247]:
# RDBにmail_dfのテーブルを作成する
entity_df = pd.DataFrame(entity_rows, columns=['message_id','entity'])

from db import connect
engine = connect()

entity_df.to_sql(name='wiki_research_l_entity',con=engine,if_exists='replace',index=None)

In [113]:
# 代名詞YOUに関する辞書
refer_you = {}
for mail in bodies:
    message_id = mail['message_id']
    # triple中の代名詞youの候補の辞書を作るために，Greetingsの行からYouの候補を取り出す．
    for greetings in mail['greetings']:
        if greetings != []:
            doc = nlp(greetings[0])
            for ent in doc.ents:
                if ent.type in ['PERSON']:
                    refer_you[message_id] = {"YOU":ent.text, "YOUR":ent.text + '\'s'}

In [114]:
# その他の代名詞I, MY, ME, WE, OUR, USに関する辞書
refer_pronoun = {}
for values in mail_df.values:
    message_id = values[1]
    sender = values[2]
    start = re.search(r'(\(.+\))',sender).start()
    end = re.search(r'(\(.+\))',sender).end()
    sender = sender[start+1:end-1]
    refer_pronoun[message_id]={'I':sender, 'MY':sender + '\'s', 'ME':sender, 'WE':sender, 'OUR':sender + '\'s', 'US':sender}

# 2つの辞書を結合する
for key, value in refer_you.items():
    if key in refer_pronoun:
        d = refer_pronoun[key]
        d.update(value)
        refer_pronoun[key] = d
    else:
        refer_pronoun[key] = value

# 文章からトリプルを抽出する

In [60]:
sentence_list = []
for mail in bodies:  #1通ずつ取り出す
    text = ''
    # 1行{行番号:文}ずつ取り出し，複数文が含まれた1つの文の連なりにする
    for sentence in mail['sentence']: 
        text = text + list(sentence.values())[0] + ' '
    # 文章を文に分解する
    if text == '':
        pass
    else:
        doc = nlp(text)
        for sentence in doc.sentences:
            # 単語のlemmatizationを辞書にする
            lemma = {}
            for word in sentence.words:
                lemma[word.text] = word.lemma
            sentence_list.append((mail['message_id'], sentence.text, lemma))

In [127]:
total_start = time.time()
# MinIEにかける
import requests
import json

extractions_list = []
for tpl in sentence_list:
    message_id = tpl[0]
    sentence = tpl[1].encode('utf-8')
    lemma = tpl[-1]
    try:
        response = requests.post('http://localhost:8080/minie/query', data=sentence)
        result = response.json()
        if result['facts'] == []:
            pass
        else:
            for triple in result['facts']:
                sbj = triple['subject']
                obj = triple['object']
                lemma_predicate = triple['predicate']
#                token_list = triple['predicate'].split()
#                lemma_list = []
#                for token in token_list:
#                    try:
#                        lemma_list.append(lemma[token])
#                    except:
#                        lemma_list.append(token)
#                lemma_predicate = ' '.join(lemma_list)
                extractions_list.append([message_id, sentence.decode('utf-8'), sbj, lemma_predicate, obj])
    except json.JSONDecodeError:
        pass

# dataframeにする
triple_df = pd.DataFrame(extractions_list, columns = ['message_id','sentence', 'subject', 'predicate', 'object'])

total_time = time.time() - total_start
print("Total time:{} minutes".format(total_time/60))

Total time:0.5135371287663778 minutes


# トリプルを代名詞の辞書と照合し，代名詞が含まれていれば対応するエンティティで置き換える

In [128]:
# Senderに置き換えるための辞書を使って実際に置き換える
triples = []
for row in triple_df.values:
    message_id = row[0]
    sentence = row[1]
    sbj = row[2].split()
    pred = row[3].split()
    obj = row[4].split()
    pronouns = refer_pronoun[message_id]
    # subjectの置き換え
    for i, word in enumerate(sbj):
        entity = pronouns.get(word.upper())
        if entity is None:
            continue
        else:
            sbj[i] = entity
    sbj = ' '.join(sbj)
    # predicateの置き換え
    for i, word in enumerate(pred):
        entity = pronouns.get(word.upper())
        if entity is None:
            continue
        else:
            pred[i] = entity
    pred = ' '.join(pred)
    # objectの置き換え
    for i, word in enumerate(obj):
        entity = pronouns.get(word.upper())
        if entity is None:
            continue
        else:
            obj[i] = entity
    obj = ' '.join(obj)
    # [new_arg1, new_rel, new_arg2s]を1行として追加
    triples.append((message_id, sentence, sbj, pred, obj))

# トリプルの主語，目的語が代名詞のみの場合，そのトリプルを除外（削除）する

# トリプルの述語に"..."が含まれている場合，そのトリプルを削除する

# トリプルの述語の一部をlemmatizationする

# トリプルとEntityの辞書を照合し，辞書に登録されているEntityで置き換える

In [129]:
canonical_triples = []
mentions = list(integrated_entity_dict.keys())
for row in triples:
    message_id = row[0]
    sentence = row[1]
    sbj = row[2]
    pred = row[3]
    obj = row[4]

    # subjectあるいはobjectが辞書に登録されているメンションと一致する場合
    if sbj in mentions:
        sbj = list(integrated_entity_dict[sbj].keys()) #メンションに対応するEntityで置き換える
        sbj = sbj[0]
        canonical_triples.append((message_id, sentence, sbj, pred, obj))
    elif obj in mentions:
        obj = list(integrated_entity_dict[obj].keys()) #メンションに対応するEntityで置き換える
        obj = obj[0]
        canonical_triples.append((message_id, sentence, sbj, pred, obj))
    else:
        for mention in mentions:
            entity = list(integrated_entity_dict[mention].keys())
            entity = entity[0]
            if re.search(re.escape(mention), sbj):
                canonical_triples.append((message_id, sentence, sbj, pred, obj))
                canonical_triples.append((message_id, sentence, entity, 'seeAlso', sbj))
            elif re.search(re.escape(mention), obj):
                canonical_triples.append((message_id, sentence, sbj, pred, obj))
                canonical_triples.append((message_id, sentence, entity, 'seeAlso', obj))

canonical_triple_df = pd.DataFrame(canonical_triples, columns=['message_id', 'sentence', 'sbject', 'predicate', 'object'])

# RDBにtripleテーブルを作る

In [130]:
from db import connect
engine = connect()

canonical_triple_df = canonical_triple_df.drop_duplicates()
canonical_triple_df.to_sql(name='wiki_research_l_triple',con=engine,if_exists='replace',index=None)

In [125]:
canonical_triple_df['predicate'].unique()

array(['has', 'be great suggestion to include', 'seeAlso', 'be include',
       'be familiar with', 'love to interview you for',
       'love test visualization for', 'love to interview', 'love',
       'schedule', 'be look for researcher so unfortunately in',
       'be look for developer so unfortunately in', 'be look for', 'be',
       'be to bring together',
       'be to bring together QUANT_O_1 party work toward',
       'invite submission of', 'talk',
       'organize experience Wikipedians to thank thousand of',
       'find that receive in', 'find that',
       'increase QUANT_R_1 week retention by',
       'increase QUANT_R_1 week retention on', 'increase', 'look at',
       'do find', 'feel', 'has effort on',
       'make valuable discovery about who spend time support other because of',
       'value', 'move pre-prints toward submission for', 'move', 'is',
       'has team of', 'be grateful to',
       'partner with J. Nathan Matias in', 'partner with', 'has in',
       'wo